In [8]:
file = open('input16.txt')

transmission = file.readline()
transmission = transmission.rstrip()
transmission = str(bin(int(transmission,16)))[2:] #convert to binary and eliminate the first two bit indicating the binary rep.


In [13]:
class Unpacked:
    
    def __init__(self,i,v,t,n,subpackets):
        
        self.i = i
        self.v = v
        self.t = t
        self.n = n
        self.subpackets = subpackets

def Unpack(packet, c):
    
    i = c
    V = int(packet[c:c+3],2)
    T = int(packet[c+3:c+6],2)
    
    c += 6
    
    if T == 4:

        return UnpackLiteral(packet,c,V,T)
    
    else:

        return UnpackOperator(packet,c,V,T)
    

def UnpackLiteral(packet,c,V,T): 
    
    i = c
    ignore = 0
    literal = str()
        
    while ignore == 0:

        a = packet[c]
        literal += packet[c+1:c+5]

        if a == '1':

            c += 5

        elif a == '0':

            ignore = 1
            c += 5
    
    l = len(literal)
    
    while l%4 != 0: #move the cursore at the end of the padding
        
        l += 1
        c += 1
        
    literal = int(literal,2)

    return (Unpacked(i,V,T,literal,[]), c)

def UnpackOperator(packet,c,V,T):
    
    I = packet[c]
    i = c
    c += 1
    
    if I == '0':

        L = int(packet[c:c+15],2)
        c += 15
        c_start = c
        
        subpackets = []
        
        while c < c_start + L:
            
            (s,c) = Unpack( packet, c )
            subpackets.append(s)

        return (Unpacked(i,V,T,len(subpackets),subpackets), c)
    
    elif I == '1':
        
        N = int(packet[c:c+11],2)
        c += 11
        
        subpackets = []
        
        while len(subpackets) < N:
            
            (s,c) = Unpack( packet, c ) 
            subpackets.append(s)
            
        return (Unpacked(i,V,T,len(subpackets),subpackets), c)
            

def loopversion(packet):
    
    s = 0
    s += packet.v
    
    subpackets = packet.subpackets
    
    if subpackets != []:
        
        for p in subpackets:
        
            s += loopvalue(p)
    
    return s
    

def Evaluate(packet):
    
    t = packet.t
    
    if t == 0:
        
        v = 0
        
        for subpacket in packet.subpackets:
            
            v += Evaluate(subpacket)
    
    if t == 1:
        
        v = 1
        
        for subpacket in packet.subpackets:
            
            v *= Evaluate(subpacket)
            
    if t == 2:
        
        vs = []
        
        for subpacket in packet.subpackets:
            
            vs.append(Evaluate(subpacket))
            
        v = min(vs)
            
    if t == 3:
        
        vs = []
        
        for subpacket in packet.subpackets:
            
            vs.append(Evaluate(subpacket))
            
        v = max(vs)
            
    if t == 4:
        
        v = packet.n
        
    if t == 5:
        
        v = 0
        
        if Evaluate(packet.subpackets[0]) > Evaluate(packet.subpackets[1]):
            
            v = 1
            
    if t == 6:
        
        v = 0
        
        if Evaluate(packet.subpackets[0]) < Evaluate(packet.subpackets[1]):
            
            v = 1
            
    if t == 7:
        
        v = 0
        
        if Evaluate(packet.subpackets[0]) == Evaluate(packet.subpackets[1]):
            
            v = 1
        
    return v
    
    
    
    

In [14]:
c = 0
(p,c) = Unpack(transmission,c)

In [15]:
print('The sum of all the version of the subpackets is ' + str(loopversion(p)))
print('The value of the outermost packet is ' + str(Evaluate(p)))

The sum of all the version of the subpackets is 1007
The value of the outermost packet is 834151779165


In [222]:
Evaluate(p)

'0'